# Asking ChatBot about chosen information
Source: https://towardsdatascience.com/lets-build-an-intelligent-chatbot-7ea7f215ada6

## Libraries or concepts used in the process
 
*   NLTK - a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning, wrappers for industrial-strength NLP libraries, and an active discussion forum.

*   TF-IDF

*   Cosine similarity

*   WordNet

*   WordNetLemmatizer

*   Wikipedia



## Installs libraries

In [2]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=f51e1ef30774636067b185fd53ac0249945ff766c8bf9273c7de69f5b1a55b51
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


## Imports libraries

In [20]:
import nltk
import random
import string
import nltk
nltk.download('averaged_perceptron_tagger')
import re, string, unicodedata
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import wikipedia as wk
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
nltk.download('punkt') 
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Opens the text file

In [0]:
data = open('/content/text_for_chatbot','r',errors = 'ignore')
raw = data.read()
raw = raw.lower()

## Shows the beginning of the file

In [7]:
raw[:100]

'human resource management is the process of recruiting, selecting, inducting employees, providing or'

## Sentence tokenizer

In [0]:
sent_tokens = nltk.sent_tokenize(raw)

In [0]:
sent_tokens[:5]

## Text normalisation

*   Word tokenization - Tokenization is the act of breaking up a sequence of strings into pieces such as words, keywords, phrases, symbols and other elements called tokens. Tokens can be individual words, phrases or even whole sentences. In the process of tokenization, some characters like punctuation marks are discarded.

*   Remove ascii - 
*   Remove tags
*   pos tagging and lemmatization

In [0]:
def Normalize(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    #word tokenization
    word_token = nltk.word_tokenize(text.lower().translate(remove_punct_dict))
    #remove ascii
    print(word_token[:100])
    new_words = []
    for word in word_token:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    print(new_word[:100])
    #Remove tags
    rmv = []
    for w in new_words:
        text=re.sub("&lt;/?.*?&gt;","&lt;&gt;",w)
        rmv.append(text)
        
    #pos tagging and lemmatization
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    lmtzr = WordNetLemmatizer()
    lemma_list = []
    rmv = [i for i in rmv if i]
    for token, tag in nltk.pos_tag(rmv):
        lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
        lemma_list.append(lemma)
    return lemma_list

In [11]:
Normalize(sent_tokens)


AttributeError: ignored

In [23]:
TfidfVec = TfidfVectorizer(tokenizer=Normalize, stop_words='english')
TfidfVec.fit_transform(sent_tokens)

['i']
i
['anyway']
anyway
['on']
on
['empty']
empty
['he']
he
['hereby']
hereby
['can', 'not']
not
['his']
his
['should']
should
['others']
others
['indeed']
indeed
['your']
your
['thereafter']
thereafter
['now']
now
['whereupon']
whereupon
['whereby']
whereby
['moreover']
moreover
['along']
along
['hereafter']
hereafter
['mill']
mill
['may']
may
['however']
however
['still']
still
['thence']
thence
['several']
several
['beforehand']
beforehand
['me']
me
['third']
third
['towards']
towards
['bottom']
bottom
['seemed']
seemed
['back']
back
['before']
before
['if']
if
['so']
so
['the']
the
['through']
through
['un']
un
['afterwards']
afterwards
['ie']
ie
['ourselves']
ourselves
['first']
first
['during']
during
['ever']
ever
['anyhow']
anyhow
['some']
some
['de']
de
['everyone']
everyone
['himself']
himself
['etc']
etc
['this']
this
['down']
down
['many']
many
['other']
other
['becomes']
becomes
['neither']
neither
['never']
never
['fifty']
fifty
['were']
were
['seem']
seem
['someone']
s

<472x1970 sparse matrix of type '<class 'numpy.float64'>'
	with 6496 stored elements in Compressed Sparse Row format>

## Creating greeting responses

In [0]:
welcome_input = ("hello", "hi", "greetings", "sup", "what's up","hey",)
welcome_response = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def welcome(user_response):
    for word in user_response.split():
        if word.lower() in welcome_input:
            return random.choice(welcome_response)
        else:
            return "Sorry, I do not recognize this greeting"

## Generating response for the knowledge question

In [0]:
def generateResponse(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=Normalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    #vals = cosine_similarity(tfidf[-1], tfidf)
    vals = linear_kernel(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0) or "tell me about" in user_response:
        print("Checking Wikipedia")
        if user_response:
            robo_response = wikipedia_data(user_response)
            return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response#wikipedia search
def wikipedia_data(input):
    reg_ex = re.search('tell me about (.*)', input)
    try:
        if reg_ex:
            topic = reg_ex.group(1)
            wiki = wk.summary(topic, sentences = 3)
            return wiki
    except Exception as e:
            print("No content has been found")

## Running the bot while True

In [17]:
flag=True
print("My name is Chatterbot and I'm a chatbot. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response not in ['bye','shutdown','exit', 'quit']):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Chatterbot : You are welcome..")
        else:
            if(welcome(user_response)!=None):
                print("Chatterbot : "+welcome(user_response))
            else:
                print("Chatterbot : ",end="")
                print(generateResponse(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Chatterbot : Bye!!! ")

My name is Chatterbot and I'm a chatbot. If you want to exit, type Bye!


KeyboardInterrupt: ignored